In [1]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
import copy
from numpy.linalg import multi_dot
from scipy.stats import norm
from scipy.stats import bernoulli
from scipy.optimize import fmin_slsqp as min

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.api import OLS

from sklearn.linear_model import LassoCV
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import AlphaSelection

from scipy.interpolate import CubicSpline

import matplotlib.pyplot as plt
import math


#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data

#import traded options parameters and info
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_strikes, generate_mthly_strikes

#Import pricing functions
from ipynb.fs.full.user_defined_vik_functions import BSM_call_vec_with_div, BSM_put_vec_with_div

#Import implied volatility processing functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_iv, gen_bivariate_interpolated_iv

#Import Risk-free interest rate generation functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_ir

#Import Stochatic process simulation functions
from ipynb.fs.full.user_defined_vik_functions import gbm1W_simulation, generate_covariance_from_correlation

# Import risk free interest rate function
from ipynb.fs.full.user_defined_vik_functions import get_risk_free_rate_from_exact_date


In [17]:
# Stock Index of Interest
stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "OTM"

#Product type to hedge: either "CE" or "PE"
prod_type = "PE"
# prod_type = "CE"

# The scope of this code is to hedge one option 
# and scope will be extended to a portfolio
no_of_assets = 1
cor_mat = [[1]]

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path

# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}

#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]

#Simulation Parameters
no_of_paths = 5000
no_opt=1


In [18]:
#########################
#User-defined functions #
#########################

def generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, \
                                       dict_wkly_expiries_each_month, \
                                       dict_ce_wkly_strikes, dict_pe_wkly_strikes, \
                                       dict_wkly_spots, dict_wkly_iv, \
                                       prod_type, prod_moneyness, output_path, \
                                       option_type = "CE", no_of_assets=1, cor_mat=[[1]],\
                                       stock_ident="BANKNIFTY"):
    ce_pfl_weights = []
    pe_pfl_weights = [] 
    cash_weights = []
    for each_month in mthly_expiries_list:
        strike = dict_mthly_strikes[each_month.strftime("%d-%b-%Y")]
        weekly_pfl_weights = []
        ce_dict = {}
        pe_dict = {}
        cash_dict = {}
        week_list = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")]
        no_of_weeks = len(week_list)
        
        for week in range(0, no_of_weeks):
            s_t = dict_wkly_spots[each_month.strftime("%d-%b-%Y")][week]
            K = strike
            r = np.array([dict_wkly_ir[each_month.strftime("%d-%b-%Y")][week]])
            
            if (week == no_of_weeks-1):
                sim_start_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week]
                sim_end_date = each_month
            else:
                sim_start_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week]
                sim_end_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week + 1]

            df_full_iv = pd.read_csv(output_path + "A3_" + stock_ident + "_implied_vol_surface.csv")
            df_date = df_full_iv[df_full_iv['Date'] == sim_start_date.strftime("%d-%b-%Y")]
            
            # We define Moneyness as (S/K) in vol surface for both calls and puts
            option_moneyness = float(s_t/float(K))
            sim_moneyness = 1.0
            
            # Vol Surface is already sorted when created in code A3 - This is just to make sure again
            df_date = df_date.sort_values(['T', 'Moneyness'], ascending=[False, True]).reset_index()
            
            # np.unique also sorts the array - we need to re-sort in descending order
            # This is required as last week has only one tenor and when filtering, we need to filter as 
            # tenor_list[0]
            
            t_list = np.sort(np.unique(np.array(df_date['T'])))
            tenor_list = [str(int(round(t* 365))) + "D" for t in t_list]
            print(each_month, " Week: ", week)
            print(tenor_list)
            
            if (len(tenor_list) > 2):
                print("There are more than 2 tenors in the vol surface - Please check!")
                
            df_tenor = df_date[df_date['Tenor'] == tenor_list[0]]

            x = np.array(df_tenor["Moneyness"])
            y = np.array(df_tenor["Impl_Vol"])
            cubic_spline = CubicSpline(x, y, bc_type='natural')
            x_min = np.amin(x)
            x_max = np.amax(x)
            
            if (sim_moneyness <= x_min):
                vol_list = [cubic_spline(x_min)]
            elif (sim_moneyness >= x_max):
                vol_list = [cubic_spline(x_max)]
            else:
                vol_list = [cubic_spline(sim_moneyness)]
                   
            sim_stock_mat = gbm1W_simulation(no_of_paths, sim_start_date, sim_end_date, \
                                             no_of_assets, s_t, r, vol_list, cor_mat)
            
            call_strikes = np.sort(np.array(dict_ce_wkly_strikes[each_month.strftime("%d-%b-%Y")][week])).reshape(1,-1)
            put_strikes = np.sort(np.array(dict_pe_wkly_strikes[each_month.strftime("%d-%b-%Y")][week])).reshape(1,-1)
            no_of_calls = call_strikes.size
            no_of_puts = put_strikes.size
            no_ind_vars = no_of_calls + no_of_puts
            
            r_f = get_risk_free_rate_from_exact_date(sim_start_date)
            stock_vec = sim_stock_mat[:, 1]
            dt = float((each_month - sim_end_date).days) / 365
            opt_strike = strike
    
            spot_list = sim_stock_mat[:,1]
            mny_list = spot_list / K

            # Linear interpolation in varaince space across tenors
            if (week != no_of_weeks-1):
                
                # Time variables
                t1 = t_list[0]
                t2 = t_list[1] 
                t_star = dt
                
                # Vol at larger tenor: corresponding to maturity of option to be hedged
                df_tenor = df_date[df_date['Tenor'] == tenor_list[1]]
                x = np.array(df_tenor["Moneyness"])
                y = np.array(df_tenor["Impl_Vol"])
                cubic_spline_mth = CubicSpline(x, y, bc_type='natural')
                x_min = np.amin(x)
                x_max = np.amax(x)
                
                vol_t2 = cubic_spline_mth(mny_list)
                vol_t2[mny_list <= x_min] = cubic_spline_mth(x_min)
                vol_t2[mny_list >= x_max] = cubic_spline_mth(x_max)
                #Variance corresponding to tenor t2
                var_t2 = np.square(vol_t2)
                
                #Vol at weekly tenor: corresponding to maturity of options used to hedge
                df_tenor = df_date[df_date['Tenor'] == tenor_list[0]]
                x = np.array(df_tenor["Moneyness"])
                y = np.array(df_tenor["Impl_Vol"])
                cubic_spline_wk = CubicSpline(x, y, bc_type='natural')
                x_min = np.amin(x)
                x_max = np.amax(x)
                
                vol_t1 = cubic_spline_wk(mny_list)
                vol_t1[mny_list <= x_min] = cubic_spline_wk(x_min)
                vol_t1[mny_list >= x_max] = cubic_spline_wk(x_max) 
                #Variance corresponding to tenor t1
                var_t1 = np.square(vol_t1)
                
                #Linear interpolation in variance space
                var_tstar = var_t1 + (var_t2 - var_t1) * ((t_star-t1)/(t2-t1))
                vol_arr = np.sqrt(var_tstar)
            else:
                vol_arr = np.array([0]) 

            if (option_type == "CE" and week != no_of_weeks-1):
                option_value = BSM_call_vec_with_div(stock_vec, opt_strike, r, r_f, vol_arr, dt)
            elif (option_type == "PE" and week != no_of_weeks-1):
                option_value = BSM_put_vec_with_div(stock_vec, opt_strike, r, r_f, vol_arr, dt) 
            elif (option_type == "CE" and week == no_of_weeks-1):
                option_value = np.maximum(stock_vec - opt_strike, 0) 
            else:
                option_value = np.maximum(opt_strike - stock_vec, 0)
    
            option_value = np.asarray(option_value)
            option_value = option_value.reshape(-1, 1) 
            
            y = option_value
            stock_vec = stock_vec.reshape(-1,1)
            x = np.concatenate((np.maximum(stock_vec-call_strikes,0), np.maximum(put_strikes-stock_vec,0)), axis=1) 

            print("SKLearn Linear regression - Lasso")
            lin_model_lasso = Lasso(alpha=1.0).fit(x, y)
            pfl_weights = lin_model_lasso.coef_
            pfl_weights = pfl_weights.reshape(-1)
            cash_weight = lin_model_lasso.intercept_
            print("R-Square: ", lin_model_lasso.score(x, y))
            print("Coefficients:")
            print(pfl_weights)
            print(cash_weight)

#             #Find Optimal Alpha
#             alphas = np.array([-1.0, -0.5, 0.0, 0.25, 0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0])
#             model = LassoCV(alphas=alphas)
#             visualizer = AlphaSelection(model)
#             visualizer.fit(x, y)
#             visualizer.show()    

            # Call Pfl weights
            call_strikes = call_strikes.reshape(-1) 
            ce_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_calls)]).reshape(-1)
            ce_wt_list = pfl_weights[0:no_of_calls]
            ce_dict = {"Date": ce_dates_list, "Strike": call_strikes, "Pfl_weights": ce_wt_list}
            df_ce = pd.DataFrame(ce_dict)
            ce_pfl_weights.append(df_ce)
    
            
            #Put Pfl Weights
            put_strikes = put_strikes.reshape(-1) 
            pe_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_puts)]).reshape(-1)
            pe_wt_list = pfl_weights[no_of_calls:no_ind_vars]
            pe_dict = {"Date": pe_dates_list, "Strike":put_strikes, "Pfl_weights": pe_wt_list}
            df_pe = pd.DataFrame(pe_dict)
            pe_pfl_weights.append(df_pe)
            
            # Cash component
            cash_date = [sim_start_date.strftime("%d-%b-%Y")]
            cash_dict = {"Date":cash_date, "Pfl_weights":cash_weight} 
            df_cash = pd.DataFrame(cash_dict)
            cash_weights.append(df_cash)

            
        df_ce_pfl_weights = pd.concat(ce_pfl_weights, axis=0)
        df_pe_pfl_weights = pd.concat(pe_pfl_weights, axis=0)
        df_cash_pfl_weights = pd.concat(cash_weights, axis=0)
    
        df_ce_pfl_weights.to_csv(output_path  + "B8A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_CE_Pfl_weights.csv", index = False)
        df_pe_pfl_weights.to_csv(output_path + "B8A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_PE_Pfl_weights.csv", index = False)
        df_cash_pfl_weights.to_csv(output_path  + "B8A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_Cash_Pfl_weights.csv", index = False)
        
    return df_ce_pfl_weights, df_pe_pfl_weights, df_cash_pfl_weights

def output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_path):
    list_of_dates = []
    list_of_prod_type = []
    list_of_strikes = []
    for each_month in dict_mthly_strikes.keys():
        list_of_dates.append(each_month)
        list_of_strikes.append(dict_mthly_strikes[each_month])
        list_of_prod_type.append(prod_type)
    dict_hedged_prod = {"Date":list_of_dates, "prod_type":list_of_prod_type, "Strike":list_of_strikes}
    df_hedged_prod = pd.DataFrame(dict_hedged_prod) 
    df_hedged_prod.to_csv(output_path + "B8A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_strikes_hedged.csv", index = False)
    return(df_hedged_prod)


In [19]:

#Find the monthly strikes of option from mkt data to find the option to be hedged
#Every month, an option is hedged
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)

#Load all monthly mkt data
mthly_mkt_data = load_all_mthly_data(mthly_expiries_list, input_data_path, holidays_list, prod_type_lists=["FUT", "CE", "PE"], stock_ident=stock_ident)

#Load weekly strikes of short term options used for hedging monthly options 
dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots = generate_weekly_strikes(dict_wkly_expiries_each_month, input_data_path, stock_ident = stock_ident)     

#Load monthly strikes of interest as of required moneyness on the begining of month
dict_mthly_strikes = generate_mthly_strikes(mthly_mkt_data, prod_moneyness, prod_type, holidays_list, stock_ident=stock_ident)

#Load weekly IV for Black-Scholes model input and monte-carlo simulation of stock paths
dict_wkly_iv = generate_weekly_iv(dict_wkly_expiries_each_month, dict_wkly_spots, dict_mthly_strikes, prod_type, output_path=output_data_path, atm_ind=0, stock_ident=stock_ident)

#Load weekly risk free interest rates from futures
#This function to be changed based on updates on risk-free-interest - now kept it as cosntant
dict_wkly_ir = generate_weekly_ir(dict_wkly_expiries_each_month, mthly_mkt_data, stock_ident)

#Load Static portfolio weights built by neural network
ce_pfl_wts, pe_pfl_wts, cash_pfl_wts = generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, dict_wkly_expiries_each_month, \
                                                        dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots, \
                                                        dict_wkly_iv, prod_type, prod_moneyness, output_data_path, \
                                                        option_type = prod_type, no_of_assets=no_of_assets, \
                                                        cor_mat=cor_mat, stock_ident=stock_ident) 
# The strike correspnding to the option that is hedged
df_hedged_prod = output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_data_path)


2019-08-29  Week:  0
['7D', '35D']
SKLearn Linear regression - Lasso
R-Square:  0.9990312814719754
Coefficients:
[-0.00103672 -0.01499396 -0.         -0.          0.          0.
  0.00375435  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.01229631]
[7.91208396]
2019-08-29  Week:  1
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9980095220793592
Coefficients:
[-2.36923976e-02 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  1.46494784e-03  6.59706346e-03  5.32533011e-03  8.63415090e-05
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  9.54582334e-03  2.56674594e-02
  2.04283127e-02  0.00000000e+00  5.77267742e-03  4.99434345e-03
  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.12206087e-02]
[9.95775198]
2019-08-29  Week:  2
['6D', '21D']
SKLearn Line

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


2019-10-31  Week:  0
['7D', '35D']
SKLearn Linear regression - Lasso
R-Square:  0.9991866301447703
Coefficients:
[-0.01227329 -0.00143338 -0.00012086 -0.         -0.         -0.
 -0.00107886 -0.         -0.         -0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.03521005  0.          0.          0.
  0.00581723  0.00528946  0.00194322  0.00925799  0.00362949  0.00397651
  0.00353784]
[17.90807008]
2019-10-31  Week:  1
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9994802614759426
Coefficients:
[-0.01824594 -0.00196043 -0.         -0.          0.          0.
  0.00229351  0.          0.          0.          0.          0.0012877
  0.          0.          0.          0.          0.02712578  0.0840703
  0.03431054  0.03238402  0.02746185  0.00764691  0.01195634  0.00467567
  0.00391245  0.01513453  0.0062463   0.0083463 ]
[18.75941475]
2019-10-31  Week:  2
['7D', '21D']
SKLearn Linear regression - Lass

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


R-Square:  1.0
Coefficients:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.]
2019-11-28  Week:  0
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9979710189431179
Coefficients:
[-4.49976884e-03 -1.15821090e-03 -2.75344751e-06 -0.00000000e+00
 -0.00000000e+00 -5.09365516e-03 -4.54088951e-03 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.46124897e-03  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  8.17605667e-03  1.98619631e-02
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2.63867523e-03  1.82174122e-03  6.38379992e-03  5.67882162e-03
  5.28040667e-03  2.21387281e-03]
[11.28739742]
2019-11-28  Week:  1
['7D', '21D']
SKLearn Linear regression - Lasso
R-Square:  0.9941625073914526
Coefficients:
[-5.55692846e-04 -9.64161007e-05 -0.00000000e+00 

...

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


2020-07-30  Week:  0
['7D', '35D']
SKLearn Linear regression - Lasso


/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1081.0290210746418, tolerance: 1034.0688371757044
  positive)


R-Square:  0.9999615961032832
Coefficients:
[-8.56171878e-02 -0.00000000e+00 -0.00000000e+00  7.35671408e-03
  9.31606131e-03  4.39567012e-03  6.18583983e-06  7.18623874e-04
  4.69332085e-03  6.54872809e-03  6.24660560e-03  5.20434635e-03
  3.77867448e-03  3.87267672e-03  9.04537751e-03  0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  5.86194188e-02  3.20198994e-02  9.92191754e-02
  6.66063785e-02  7.13540926e-03  2.08901643e-02  4.13401891e-02
  1.52043306e-02  0.00000000e+00 -0.00000000e+00  3.26556426e-02
  7.92763850e-02  2.31536940e-02  3.02939243e-03]
[80.29624092]
2020-07-30  Week:  1
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9996164506967399
Coefficients:
[-0.02009964 -0.0009064  -0.         -0.          0.          0.
  0.         -0.         -0.00238828 -0.         -0.          0.
  0.00638601  0.00109765  0.          0.          0.          0.

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
